### *IPCC SR15 scenario assessment*

<img style="float: right; height: 80px; padding-left: 20px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Analysis of radiative forcing from short-lived climate-forcers

This notebook plots the radiative forcing from short-lived climate forcers (SLCF)
shown in **Figure 2.8** of the IPCC's _"Special Report on Global Warming of 1.5°C"_.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load `pyam` package and other dependencies

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
plt.style.use('style_sr15.mplstyle')
%matplotlib inline
import pyam

from utils import boxplot_by_cat

## Import scenario data, categorization and specifications files

The metadata file must be generated from the notebook `sr15_2.0_categories_indicators` included in this repository.  
If the snapshot file has been updated, make sure that you rerun the categorization notebook.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r2.0.xlsx')

In [ ]:
sr1p5.load_meta('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream, Loader=yaml.FullLoader)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
all_cats = specs.pop('all_cats')
subcats = specs.pop('subcats')
all_subcats = specs.pop('all_subcats')
plotting_args = specs.pop('plotting_args')
marker= specs.pop('marker')

## Downselect scenario ensemble to categories of interest for this assessment

In [ ]:
years = [2010, 2020, 2030, 2050, 2100]

In [ ]:
cats.remove('Above 2C')

In [ ]:
sr1p5.meta.rename(columns={'Kyoto-GHG|2010 (SAR)': 'kyoto_ghg_2010'}, inplace=True)

In [ ]:
df = sr1p5.filter(kyoto_ghg_2010='in range', category=cats, year=years)

## Set specifications for filter and plotting

In [ ]:
save_name = 'output/fig2.8_effective_radiative_forcing.{}'

In [ ]:
filter_args = dict(df=sr1p5, category=cats, marker=None, join_meta=True)

In [ ]:
def plotting_args(filetype='png'):
    return {'categories': cats, 'column': 'category',
            'years': years, 'add_marker': marker,
            'save': save_name.format(filetype)}

## Compute radiative forcing from short-lived climate forcers

In [ ]:
df.filter(variable='AR5 climate diagnostics|Forcing|CO2|*').variables()

In [ ]:
rf_llcf_vars = [
    'AR5 climate diagnostics|Forcing|CO2|FAIR|MED',
    'AR5 climate diagnostics|Forcing|N2O|FAIR|MED'
]

In [ ]:
for v in ['AR5 climate diagnostics|Forcing|FAIR|MED'] + rf_llcf_vars:
    df.require_variable(v, exclude_on_fail=True)

df.filter(exclude=False, inplace=True)

In [ ]:
rf_all = df.filter(variable='AR5 climate diagnostics|Forcing|FAIR|MED').timeseries()
rf_all.index = rf_all.index.droplevel([2, 3, 4])

In [ ]:
rf_llcf = df.filter(variable=rf_llcf_vars).timeseries()
rf_llcf = rf_llcf.groupby(pyam.META_IDX).sum()

In [ ]:
rf_slcf = rf_all - rf_llcf

In [ ]:
data = pyam.filter_by_meta(rf_slcf, **filter_args)
boxplot_by_cat(data, **plotting_args(),
               ylabel='Effective radiative forcing (W/m2)', )

## Export timeseries data to `xlsx`

In [ ]:
data.reset_index().to_excel('output/fig2.8_data_table.xlsx')